**Dataset:**
personality type data (Lab 2 - Personality Profile Type.csv)

**Objective:**
classify Personality type as one of the following **using RNNs**. <br>
['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']

**Evaluation metric:**
Precision

### Import used libraries

In [25]:
!pip install nltk

In [23]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
from sklearn.model_selection import train_test_split
import random
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense
from keras.layers import Bidirectional, Dropout
from sklearn.metrics import precision_score



### Load Dataset

In [2]:
df = pd.read_csv("Lab 2 - Personality Profile Type.csv")
df.head(10)

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments https://www.youtube.com/watch?v=iz7lE1g4XM4 sportscenter not top ten plays https://www.youtube.com/watch?v=uCdfze1etec pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8 http://www.youtube.com/watch?v=u8ejam5DP3E On repeat for most of today.|||May the PerC Experience immerse you.|||The last ...
1,ENTP,'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin* That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This...
2,INTP,"'Good one _____ https://www.youtube.com/watch?v=fHiGbolFFGw|||Of course, to which I say I know; that's my blessing and my curse.|||Does being absolutely positive that you and your best friend could be an amazing couple count? If so, than yes. Or it's more I could be madly in love in case I reconciled my feelings (which at...|||No, I didn't; thank you for a link!|||So-called Ti-Si loop (and it can stem from any current topic/obsession) can be deadly. It's like when you're stuck in your o..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the universe and the idea that every rule and social code being arbitrary constructs created...|||Dear ENTJ sub, Long time no see. Sincerely, Alpha|||None of them. All other types hurt in deep existential ways that I want no part of.|||Probably a sliding scale that depends on individual preferences, like everything in humanity.|||Draco Malfoy also. I'd say he's either 358 or 368.|||I'm either 358..."
4,ENTJ,'You're fired.|||That's another silly misconception. That approaching is logically is going to be the key to unlocking whatever it is you think you are entitled to. Nobody wants to be approached with BS...|||But guys... he REALLY wants to go on a super-duper-long-ass vacation. C'mon guys. His boss just doesn't listen or get it. He even approached him logically and everything.|||Never mind. Just go on permanent vacation.|||Two months? I wouldn't be crazy about the idea. If you are really hi...
5,INTJ,"'18/37 @.@|||Science is not perfect. No scientist claims that it is, or that scientific information will not be revised as we discover new things. Rational thinking has been very useful to our society....|||INFP- Edgar Allen Poe was an INFP and he's in your siggy.|||People see the obvious Fi and are quick to put her as INFP. I agree that she has no Ne. I see her as an ISFP. Compare her to Haku (definite INFP). She is flat through most of Naruto.. but I don't...|||Lets get this party star..."
6,INFJ,"'No, I can't draw on my own nails (haha). Those were done by professionals on my nails. And yes, those are all gel. You mean those you posted were done by yourself on your own nails? Awesome!|||Probably the Electronic Screen Syndrome. With the advent of technology and social media, we all suffer from overstimulation on a daily basis. I'm guilty as well. In the past, I can be happy just...|||I love nail arts too! These are some of mine: 718282 718290 718298 718306 718314|||This is the first..."
7,INTJ,"'I tend to build up a collection of things on my desktop that i use frequently and then move them into a folder called 'Everything' from there it get sorted into type and sub type|||i ike to collect odd objects, even at work...a lot of people would call it junk but i like to collect it. Old unused software? ill take that off your hands :) i have a bunch of old adobe...|||i think its quite normal, i tend to only see my friends in real life every c

### Data splitting

In [3]:
x=df['posts']
y=df['type']

In [4]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2 , random_state=99)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(x_test.shape)

(6940,)
(1735,)
(6940,)
(1735,)


### EDA on training data

- check NaNs

In [9]:
x_train.isna().sum()

0

- check duplicates

In [11]:
x_train.duplicated().sum()

0

- show a representative sample of data texts to find out required preprocessing steps

In [18]:
sample_posts = x_train.sample(n=10, random_state=3099)  
for text in sample_posts:
    print("*****************")
    print(text)
    print("*****************")

*****************
'Dear ISTPs,   7000 posts and this doesn't qualify for a sticky?|||I'd guess that there is a correlation between how many male vs female istps there are in the gen pop and how many questions get asked about each here.|||Well, like the other NF who ressponded, apparently I've entirely missed the point as well. That's fine, as far as I'm concerned.   Look, how you are talking about this issue reads to me as though...|||While you're waiting for some ISTPs to give you their insight, I will offer this. Get his number and text him. If he responds, quickly or within the first 24 hours (sometimes they like to craft their...|||every useless bit of DNA they run into  Dear ISTP:  That is  the funniest description I have ever heard an ISTP use about other humans ever.  B|||As I've said a few times to my ISTP friend, you're not so special that you get to be the only misunderstood person on the planet.   Yeah, you're friends probably get it and accept you, otherwise I...|||Chill. I

- check dataset balancing

In [19]:
class_distribution = y_train.value_counts()

In [20]:
print("Class Distribution:")
print(class_distribution) #data is not balanced

Class Distribution:
type
INFP    1476
INFJ    1176
INTP    1025
INTJ     867
ENFP     554
ENTP     552
ISTP     269
ISFP     224
ENTJ     176
ISTJ     165
ENFJ     157
ISFJ     131
ESTP      68
ESFP      35
ESTJ      33
ESFJ      32
Name: count, dtype: int64


- Cleaning and Preprocessing are:
    - 1 removing urls
    - 2 removing special characters
    - 3 lowercasing
    - 4 removing stop words
    - 5 lemmatization

### Cleaning and Preprocessing

downloading resources 

In [6]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ITI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ITI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ITI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [8]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters, numbers, and dots
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Lowercasing
    text = text.lower()
    # Tokenization
    tokens = nltk.word_tokenize(text)
    # Removing stop words
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

In [9]:
X_train_preprocessed = x_train.apply(preprocess_text)
X_test_preprocessed = x_test.apply(preprocess_text)

In [10]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

**You  are doing Great so far!**

### Modelling

In [12]:
#tokenization
max_words = 1000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train_preprocessed)
X_train_seq = tokenizer.texts_to_sequences(X_train_preprocessed)
X_test_seq = tokenizer.texts_to_sequences(X_test_preprocessed)



In [13]:
# Padding 
max_len = 200
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [14]:
#model
model = Sequential([
    Embedding(max_words, 64),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(16, activation='softmax')
])

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=128)


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 8s 120ms/step - accuracy: 0.3156 - loss: 2.1210
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.3474 - loss: 2.0583
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 123ms/step - accuracy: 0.3665 - loss: 1.9852
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 114ms/step - accuracy: 0.3561 - loss: 1.9755
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 122ms/step - accuracy: 0.3914 - loss: 1.8999
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.4046 - loss: 1.8667
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.4244 - loss: 1.8390
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.4308 - loss: 1.7997
Epoch 9/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.4321 - loss: 1.7813
Epoch 10/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 0.4488 - loss: 1.7499


#### Evaluation

**Evaluation metric:**
Precision

In [19]:
y_pred = model.predict(X_test_pad)


55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [21]:
y_pred_classes = np.argmax(y_pred, axis=1)


In [22]:
precision = precision_score(y_test_encoded, y_pred_classes, average='weighted')
print("Precision:", precision)

Precision: 0.24674709302069783


C:\Users\ITI\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Enhancement

In [24]:
model = Sequential([
    Embedding(max_words, 128),  
    Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)),  
    Dropout(0.5),  
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),  
    Dense(32, activation='relu'),  
    Dense(16, activation='softmax')
])

In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [26]:
model.fit(X_train_pad, y_train_encoded, epochs=5, batch_size=128)


Epoch 1/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 42s 692ms/step - accuracy: 0.1845 - loss: 2.5274
Epoch 2/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 40s 723ms/step - accuracy: 0.2061 - loss: 2.2953
Epoch 3/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 42s 763ms/step - accuracy: 0.1973 - loss: 2.2954
Epoch 4/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 44s 799ms/step - accuracy: 0.2076 - loss: 2.2718
Epoch 5/5
55/55 ━━━━━━━━━━━━━━━━━━━━ 44s 794ms/step - accuracy: 0.2587 - loss: 2.2265


In [27]:
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
precision = precision_score(y_test_encoded, y_pred_classes, average='weighted')
print("Precision:", precision)

55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step
Precision: 0.1392904953514593


C:\Users\ITI\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Conclusion and final results


The precision score after enhancing the model architecture is 0.139.<br>
This indicates that the model's performance has not improved compared to the initial model (Precision: 0.246).

#### Done!